# Merging SDFITS Files

This recipe shows how to merge SDFITS files. 
There is no built-in function to merge SDFITS files, so we will put the files we want to merge in the same directory, and then load them and write them to a single SDFITS file.
For this recipe we will use a single SDFITS file to generate two files from subsets of its data.

You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/merge_sdfits.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/merge_sdfits.ipynb" download>here</a> and selecting "Save Link As".

## Loading Modules
We start by loading the modules we will use for this recipe. 

In [1]:
# These modules are required for loading and writing data.
from dysh.fits import GBTFITSLoad

# These modules are only used to download data.
from pathlib import Path
from dysh.util.download import from_url

## Data Retrieval

Download the example SDFITS data, if necessary.

In [2]:
url = "http://www.gb.nrao.edu/dysh/example_data/positionswitch/data/AGBT05B_047_01/AGBT05B_047_01.raw.acs/AGBT05B_047_01.raw.acs.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

## Data Loading

Next, we use {py:class}`GBTFITSLoad <dysh.fits.GBTFITSLoad>` to load the data, and then its {py:class}`summary <dysh.fits.GBTFITSLoad.summary>` method to inspect its contents.

In [3]:
sdfits = GBTFITSLoad(filename)

In [4]:
sdfits.summary()

SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATION
51,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,198.3431,18.6427
52,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,198.9306,18.7872
53,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,199.3305,18.3561
54,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,199.9157,18.4927
55,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,200.3042,18.0575
56,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,200.8906,18.1860
57,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,202.3275,17.3853
58,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,202.9192,17.4949


## Writing SDFITS Files

To show how to merge SDFITS files, we will create two separate SDFITS files first.
We use the {py:class}`GBTFITSLoad.write <dysh.fits.GBTFITSLoad.write>` method to write the data.
We will write scans 51 and 52 to two separate files.

### Create an Output Directory

The approach we will use to merge SDFITS files relies on having the files to be merged in the same directory.
We create a new directory ``./output/merge_sdfits`` if it doesn't exist already.

In [5]:
output_dir = Path.cwd() / "output/merge_sdfits"
output_dir.mkdir(exist_ok=True, parents=True) # Create the output directory if it does not exist.

### Write the Intermediate SDFITS Files

Now we write scans 51 and 52 to the directory we created.

In [6]:
sdfits.write(output_dir / "scan51.fits", scan=51)
sdfits.write(output_dir / "scan52.fits", scan=52)

## Merge SDFITS Files

Now that we have two SDFITS files we want to merge, we can do this by loading them using {py:class}`GBTFITSLoad <dysh.fits.GBTFITSLoad>` by passing the directory containing the files we want to merge.
In this case, that would be ``./output/merge_sdfits``.

In [7]:
sdfits_merged = GBTFITSLoad(output_dir)

Loaded 2 FITS files


In [8]:
sdfits_merged.summary()

SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATION
51,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,198.3431,18.6427
52,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,198.9306,18.7872


### Write the Merged SDFITS Files to a New File

Now we can write the merged SDFITS files to a new file using {py:class}`GBTFITSLoad.write <dysh.fits.GBTFITSLoad.write>`.
We tell it to write to a single file with the ``multifile=False`` argument.
We write to a new directory to show that this is only one file now.

In [9]:
new_dir = Path.cwd() / "output/merged_sdfits"
new_dir.mkdir(exist_ok=True)
sdfits_merged.write(new_dir / "merged_sdfits.fits", multifile=False)

Load the merged SDFITS file.

In [10]:
sdfits_merged = GBTFITSLoad(new_dir)
sdfits_merged.summary()

SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATION
51,NGC5291,4386.0,OnOff,1,1.420405,1.420405,1,2,11,1,198.3431,18.6427
52,NGC5291,4386.0,OnOff,2,1.420405,1.420405,1,2,11,1,198.9306,18.7872


You can check that it is indeed a single file by looking at the ``files`` attribute of the ``GBTFITSLoad`` object.

In [11]:
sdfits_merged.files

[PosixPath('/home/scratch/psalas/python/dysh/notebooks/examples/output/merged_sdfits/merged_sdfits.fits')]

The same principles can be applied to more complex examples, like selecting an IF for certain scans, and another for others, and then merging them all in a single file.